# **Power Price Forecasting**
## *Author*: Phani Arvind Vadali
The objective of this study is to construct a black box model to forecast the hourly electricity demand one day ahead. The country of consideration here is Spain the data for whom can be obtained from ENTSOE [1].Three years worth of data from Jan, 2020 to Dec, 2022 will be used. To keep the problem simple electricity generation by different sources has been summed up to identify one value for the total net electricity generated by all sources. Similarly, import of electricity from neighbouring countries has also been aggregated into one value.

The objective of the study here is to construct a model to predict thee prive for all 24 hours a day in advance, Therefore, there are 24 outputs which are each the forecasted value of demand for the next day. The inputs to the problem are all the values of demand, generation and import till the start of the next 24 hours. The appropriate amount of time lag has to be identified. The problem looks like the following:
$$\textbf{DF} = f(\textbf{D},\textbf{NG},\textbf{Imp})$$

Here **DF** is the vector containing 24 outputs which are the value of forecasted demand for the next 24 hours. The inputs are the demand (**D**), generation (**NG**) and the import (**Imp**) for a yet undecided number of hours prior.

All value of inputs *L* days prior are used to predict the 24 hour demand. The inputs of each day i.e., the demand, generation and import values will be flattened and taken as one 24x3 sized vector. By stacking data from several days on top of each other an input matrix can now be constructed. Therefore each in put matrix will be of size *L* by 72. Each input matrix will be used to predict the values of demand for the forthcoming 24 hours. A key fact to keep in mind is that logging of input can only start L days after the beginning of the data and has to end 1 day before the end of the dataset.

## Mount the drive

In [ ]:
from google.colab import drive  # The google colab module to access folders on GDrive
import os  # the python module for all things related to the OS.

# we mount our gDrive drive at the startpoint
drive.mount('/content/drive')

# change that into the path you want to change into (as if you were starting in your current root folder in drive)
my_folder_path = "AREN5030/HOMEWORKS/ADAM Repo"
# we navigate to the target folder
os.chdir("drive/My Drive/" + my_folder_path)

Mounted at /content/drive


## Load the packages

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import plotly.subplots as sp
import statsmodels.api as sm
import random
import itertools

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split, cross_val_score

RNN specfic packages to be imported.

In [ ]:
import torch
from torch import nn
from torch.optim import RMSprop
from torch.utils.data import DataLoader,TensorDataset

Use pip install any missing packages.

In [ ]:
%pip install torchmetrics
%pip install torchinfo
%pip install pytorch_lightning
%pip install torchvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.4/840.4 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.9/800.9 kB 5.0 MB/s eta 0:00:00


In [ ]:
from torchmetrics import MeanAbsoluteError, R2Score
import pytorch_lightning as pl
from torchinfo import summary
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import CSVLogger
pl.seed_everything(0, workers=True)
torch.use_deterministic_algorithms(True, warn_only=True)
from torchvision.transforms import (Resize,Normalize,CenterCrop,ToTensor)

INFO:lightning_fabric.utilities.seed:Seed set to 0


## Data Cleaning and Aggregation
Data for Spain from 2020 to 2023 was extracted into csv files from ENTSOE [1], cleaned, and aggregated. For the sake of brevity detailed information on various procedures undertaken is not provided here.

In [ ]:
ActGen2020 = pd.read_csv("SpainData/Actual Generation per Production Type_202001010000-202101010000.csv")
ActGen2021 = pd.read_csv("SpainData/Actual Generation per Production Type_202101010000-202201010000.csv")
ActGen2022 = pd.read_csv("SpainData/Actual Generation per Production Type_202201010000-202301010000.csv")

TotLoad2020 = pd.read_csv("SpainData/Total Load - Day Ahead _ Actual_202001010000-202101010000.csv")
TotLoad2021 = pd.read_csv("SpainData/Total Load - Day Ahead _ Actual_202101010000-202201010000.csv")
TotLoad2022 = pd.read_csv("SpainData/Total Load - Day Ahead _ Actual_202201010000-202301010000.csv")
df2020 = pd.concat([ActGen2020,TotLoad2020],axis=1)
df2021 = pd.concat([ActGen2021,TotLoad2021],axis=1)
df2022 = pd.concat([ActGen2022,TotLoad2022],axis=1)


In [ ]:
start_date_2020_1 = "2020-01-01"
end_date_2020_1 = "2020-10-25 02:00:00"
date_range_2020_1 = pd.date_range(start=start_date_2020_1, end=end_date_2020_1, freq='H')
start_date_2020_2 = "2020-10-25 02:00:00"
end_date_2020_2 = "2020-12-31 23:00:00"
date_range_2020_2 = pd.date_range(start=start_date_2020_2, end=end_date_2020_2, freq='H')

df2020_1 = df2020.iloc[:7155,:]
df2020_2 = df2020.iloc[7155:,:]
df2020_1.set_index(date_range_2020_1,inplace=True)
df2020_2.set_index(date_range_2020_2,inplace=True)
df2020 = pd.concat([df2020_1,df2020_2])

start_date_2021_1 = "2021-01-01"
end_date_2021_1 = "2021-10-31 02:00:00"
date_range_2021_1 = pd.date_range(start=start_date_2021_1, end=end_date_2021_1, freq='H')
start_date_2021_2 = "2021-10-31 02:00:00"
end_date_2021_2 = "2021-12-31 23:00:00"
date_range_2021_2 = pd.date_range(start=start_date_2021_2, end=end_date_2021_2, freq='H')

df2021_1 = df2021.iloc[:7275,:]
df2021_2 = df2021.iloc[7275:,:]
df2021_1.set_index(date_range_2021_1,inplace=True)
df2021_2.set_index(date_range_2021_2,inplace=True)
df2021 = pd.concat([df2021_1,df2021_2])

start_date_2022_1 = "2022-01-01"
end_date_2022_1 = "2022-05-23 01:00:00"
date_range_2022_1 = pd.date_range(start=start_date_2022_1, end=end_date_2022_1, freq='H')
start_date_2022_2 = "2022-05-23 02:00:00"
end_date_2022_2 = "2022-10-30 02:45:00"
date_range_2022_2 = pd.date_range(start=start_date_2022_2, end=end_date_2022_2, freq='15T')
start_date_2022_3 = "2022-10-30 02:00:00"
end_date_2022_3 = "2022-12-31 23:45:00"
date_range_2022_3 = pd.date_range(start=start_date_2022_3, end=end_date_2022_3, freq='15T')

df2022_1 = df2022.iloc[:3410,:]
df2022_2 = df2022.iloc[3410:18774,:]
df2022_3 = df2022.iloc[18774:,:]

df2022_1.set_index(date_range_2022_1,inplace=True)
df2022_2.set_index(date_range_2022_2,inplace=True)
df2022_2 = df2022_2.resample("1H").mean(numeric_only=True)
df2022_3.set_index(date_range_2022_3,inplace=True)
df2022_3 = df2022_3.resample("1H").mean(numeric_only=True)
df2022_2 = pd.concat([df2022_1,df2022_2])
df2022 = pd.concat([df2022_2,df2022_3])


In [ ]:

DayAheadPrice2020 = pd.read_csv("SpainData/Day-ahead Prices_202001010000-202101010000.csv").set_index(df2020.index)
DayAheadPrice2021 = pd.read_csv("SpainData/Day-ahead Prices_202101010000-202201010000.csv").set_index(df2021.index)
DayAheadPrice2022 = pd.read_csv("SpainData/Day-ahead Prices_202201010000-202301010000.csv").set_index(df2022.index)
df2020 = pd.concat([df2020,DayAheadPrice2020],axis=1)
df2021 = pd.concat([df2021,DayAheadPrice2021],axis=1)
df2022 = pd.concat([df2022,DayAheadPrice2022],axis=1)

CrossBorderFR2020 = pd.read_csv("SpainData/Cross-Border Physical Flow_202001010000-202101010000.csv").set_index(df2020.index)
CrossBorderFR2021 = pd.read_csv("SpainData/Cross-Border Physical Flow_202101010000-202201010000.csv").set_index(df2021.index)
CrossBorderFR2022 = pd.read_csv("SpainData/Cross-Border Physical Flow_202201010000-202301010000.csv").set_index(df2022.index)
df2020 = pd.concat([df2020,CrossBorderFR2020],axis=1)
df2021 = pd.concat([df2021,CrossBorderFR2021],axis=1)
df2022 = pd.concat([df2022,CrossBorderFR2022],axis=1)

CrossBorderPT2020 = pd.read_csv("SpainData/Cross-Border Physical Flow_202001010000-202101010000-Portugal.csv").set_index(df2020.index)
CrossBorderPT2021 = pd.read_csv("SpainData/Cross-Border Physical Flow_202101010000-202201010000-Portugal.csv").set_index(df2021.index)
CrossBorderPT2022 = pd.read_csv("SpainData/Cross-Border Physical Flow_202201010000-202301010000-Portugal.csv").set_index(df2022.index)
df2020 = pd.concat([df2020,CrossBorderPT2020],axis=1)
df2021 = pd.concat([df2021,CrossBorderPT2021],axis=1)
df2022 = pd.concat([df2022,CrossBorderPT2022],axis=1)

In [ ]:
# Not sure what to do about biomass and other sources
RenewableSources = ['Geothermal  - Actual Aggregated [MW]','Hydro Pumped Storage  - Actual Aggregated [MW]',
       'Hydro Pumped Storage  - Actual Consumption [MW]',
       'Hydro Run-of-river and poundage  - Actual Aggregated [MW]',
       'Hydro Water Reservoir  - Actual Aggregated [MW]',
       'Marine  - Actual Aggregated [MW]', 'Nuclear  - Actual Aggregated [MW]','Other renewable  - Actual Aggregated [MW]',
       'Solar  - Actual Aggregated [MW]',
       'Wind Offshore  - Actual Aggregated [MW]',
       'Wind Onshore  - Actual Aggregated [MW]']
RenSourceDict = {'Geothermal  - Actual Aggregated [MW]':"Geo",'Hydro Pumped Storage  - Actual Aggregated [MW]':"Hydro-pumped-agg",
       'Hydro Pumped Storage  - Actual Consumption [MW]':"Hydro-pumped",
       'Hydro Run-of-river and poundage  - Actual Aggregated [MW]':"Hydro Run-of-river",
       'Hydro Water Reservoir  - Actual Aggregated [MW]':"Hydro water reservoir",
       'Marine  - Actual Aggregated [MW]':"Marine", 'Nuclear  - Actual Aggregated [MW]':"Nuclear",'Other renewable  - Actual Aggregated [MW]':"Other renewable",
       'Solar  - Actual Aggregated [MW]':"Solar",
       'Wind Offshore  - Actual Aggregated [MW]':"Wind offshore",
       'Wind Onshore  - Actual Aggregated [MW]':"Wind onshore"}
NonRenewableSources = ['Fossil Brown coal/Lignite  - Actual Aggregated [MW]',
       'Fossil Coal-derived gas  - Actual Aggregated [MW]',
       'Fossil Gas  - Actual Aggregated [MW]',
       'Fossil Hard coal  - Actual Aggregated [MW]',
       'Fossil Oil  - Actual Aggregated [MW]',
       'Fossil Oil shale  - Actual Aggregated [MW]',
       'Fossil Peat  - Actual Aggregated [MW]']
NonRenSourceDict = {'Fossil Brown coal/Lignite  - Actual Aggregated [MW]':"Brown Coal",
       'Fossil Coal-derived gas  - Actual Aggregated [MW]':"Coal-dervied Gas",
       'Fossil Gas  - Actual Aggregated [MW]':"Gas",
       'Fossil Hard coal  - Actual Aggregated [MW]':"Hard Coal",
       'Fossil Oil  - Actual Aggregated [MW]':"Oil",
       'Fossil Oil shale  - Actual Aggregated [MW]':"Oil shale",
       'Fossil Peat  - Actual Aggregated [MW]':"Peat"}


# Energy coming in to Spain is negative and energy going out is positive
df2020["TIFR"] = -df2020['France (FR) > Spain (ES) [MW]'] + df2020['Spain (ES) > France (FR) [MW]']
df2021["TIFR"] = -df2021['France (FR) > Spain (ES) [MW]'] + df2021['Spain (ES) > France (FR) [MW]']
df2022["TIFR"] = -df2022['France (FR) > Spain (ES) [MW]'] + df2022['Spain (ES) > France (FR) [MW]']

df2020["TIPT"] = -df2020['Portugal (PT) > Spain (ES) [MW]'] + df2020['Spain (ES) > Portugal (PT) [MW]']
df2021["TIPT"] = -df2021['Portugal (PT) > Spain (ES) [MW]'] + df2021['Spain (ES) > Portugal (PT) [MW]']
df2022["TIPT"] = -df2022['Portugal (PT) > Spain (ES) [MW]'] + df2022['Spain (ES) > Portugal (PT) [MW]']



In [ ]:
df2020 = df2020.rename(columns={'Day-ahead Total Load Forecast [MW] - Spain (ES)':"DF",'Actual Total Load [MW] - Spain (ES)':"D",'Day-ahead Price [EUR/MWh]':"Price"})
df2021 = df2021.rename(columns={'Day-ahead Total Load Forecast [MW] - Spain (ES)':"DF",'Actual Total Load [MW] - Spain (ES)':"D",'Day-ahead Price [EUR/MWh]':"Price"})
df2022 = df2022.rename(columns={'Day-ahead Total Load Forecast [MW] - Spain (ES)':"DF",'Actual Total Load [MW] - Spain (ES)':"D",'Day-ahead Price [EUR/MWh]':"Price"})

ReqColumns=[]
ReqColumns.append(RenewableSources)
ReqColumns.append(NonRenewableSources)
ReqColumns.append(["TIFR","TIPT","DF","D","Price"])
ReqColumns = [item for sublist in ReqColumns for item in sublist]

df2020 = df2020.loc[:,df2020.columns.isin(ReqColumns)]
df2021 = df2021.loc[:,df2021.columns.isin(ReqColumns)]
df2022 = df2022.loc[:,df2022.columns.isin(ReqColumns)]


In [ ]:
df2020 = df2020.fillna(0)
df2021 = df2021.fillna(0)
df2022 = df2022.fillna(0)

In [ ]:
df2020["NetGen"] = df2020.loc[:,"Fossil Brown coal/Lignite  - Actual Aggregated [MW]":"Wind Onshore  - Actual Aggregated [MW]"].sum(axis=1)
df2021["NetGen"] = df2021.loc[:,"Fossil Brown coal/Lignite  - Actual Aggregated [MW]":"Wind Onshore  - Actual Aggregated [MW]"].sum(axis=1)
df2022["NetGen"] = df2022.loc[:,"Fossil Brown coal/Lignite  - Actual Aggregated [MW]":"Wind Onshore  - Actual Aggregated [MW]"].sum(axis=1)
df = pd.concat([df2020,df2021])
df = pd.concat([df,df2022])

## Data Preparation
Just the demand, import, generation and forecast are separated from the full dataset.

In [ ]:
df["TI"] = df[["TIFR","TIPT"]].sum(axis=1)
df = df[["D","TI","NetGen","DF"]].astype(float)
df_D_mean = df[["D"]].mean()
df_D_std = df[["D"]].std()
df_DF_mean = df[["DF"]].mean()
df_DF_std = df[["DF"]].std()

Standardize each column with respect to its respective mean and std.

In [ ]:
df = pd.DataFrame(StandardScaler(with_mean=True,with_std=True).fit_transform(df),columns=df.columns,index=df.index)
df1 = df[["D","TI","NetGen"]]

## Recurrent Neural Network Model Construction
Here an RNN model is constructed to predict the 24 hour demand forecast based on the values of demand, generation and import for few days prior. The number of lag days and the number of hidden nodes and epochs are determined using a sensitivity analysis in a later section.

The number of training days is a variable of our choice. That means that we choose how many days worth of data will be used to predict the demand in the next 24 hours. The input matrix **X** represents the value of demand, generation and import for the *L* lag days. Each input matrix will be of size L by 72. Here L is the number of lag days. Each row contains hourly values of demand, import and generation laid next to each other. Since there are 24 values each in each day the final row will have 72 elements. So the 72 input nodes will be used along with a chosen number of hidden nodes to finally output 24 values each of which are the forecast of the hourly demand in the following 24 hour period.

In [ ]:
L = 8 # Training horizon in days
Lh = L*24 # Training horizon in hours
P = 24  # Prediction horizon of 24 hours
n = df1.shape[0] - Lh - P # Total number of data presentations
# Input matrix X is a collection of n, L by 72 sized matrices
X = np.zeros((n,L,P*3))
# Outmatrix is a collection of n, P sized vectors
Y = np.zeros((P,n))
for j in range(0,n):
  for i in range(0,L):
    X[j,i,:] = df1.iloc[j+P*i:j+P+P*i,:].values.flatten(order="F")
  Y[:,j] = df1["D"].iloc[j+Lh:j+Lh+P]


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y.T, test_size=0.2, random_state=42)
X_train_rnn = torch.tensor(X_train.astype(np.float32))
y_train_rnn = torch.tensor(y_train.astype(np.float32))
X_test_rnn = torch.tensor(X_test.astype(np.float32))
y_test_rnn = torch.tensor(y_test.astype(np.float32))
DFTensorTrain = TensorDataset(X_train_rnn,y_train_rnn)
DFTensorTest = TensorDataset(X_test_rnn,y_test_rnn)

In [ ]:
class DemandForecastModel(pl.LightningModule):
  def __init__(self):
    super(DemandForecastModel, self).__init__()
    # Input units are 72 and hidden units are 256
    self.rnn = nn.RNN(3*24,256,batch_first=True)
    # Eventually we need 1 day's worth of hourly demand
    self.dense = nn.Linear(256, 24)
    # Assumed dropout rate of 10% which means that 10% of nodes are dropped out every ouptut calculation step
    self.dropout = nn.Dropout(0.005)
    self.criterion = nn.MSELoss()

  def forward(self, x):
    val, h_n = self.rnn(x)
    val = self.dense(self.dropout(val[:,-1]))
    return val

  def training_step(self, batch, batch_idx):
        x, y = batch
        predictions = self(x)
        loss = self.criterion(predictions, y)
        return loss

  def configure_optimizers(self):
      return torch.optim.Adam(self.parameters(), lr=0.001)  # Assumed learning rate of 0.001

DFModel = DemandForecastModel()

In [ ]:
# To view the final shape of the RNN
# 8 lag days with 72 elements are used to finally output 24 values
summary(DFModel, input_data=X_train_rnn, col_names=['input_size','output_size','num_params'])

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #
DemandForecastModel                      [20872, 8, 72]            [20872, 24]               --
├─RNN: 1-1                               [20872, 8, 72]            [20872, 8, 256]           84,480
├─Dropout: 1-2                           [20872, 256]              [20872, 256]              --
├─Linear: 1-3                            [20872, 256]              [20872, 24]               6,168
Total params: 90,648
Trainable params: 90,648
Non-trainable params: 0
Total mult-adds (G): 14.23
Input size (MB): 48.09
Forward/backward pass size (MB): 345.97
Params size (MB): 0.36
Estimated Total Size (MB): 394.43

In [ ]:
# Fit the model and find the weigths.
# This step will take time. Approx 11 min.
# The SGD uses a batch size of 32
DFDataLoader = DataLoader(DFTensorTrain, batch_size=32, shuffle=True,num_workers=2)
DF_trainer = pl.Trainer(deterministic=True, max_epochs=100)
DF_trainer.fit(DFModel, DFDataLoader)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type    | Params
--------------------------------------
0 | rnn       | RNN     | 84.5 K
1 | dense     | Linear  | 6.2 K 
2 | dropout   | Dropout | 0     
3 | criterion | MSELoss | 0     
--------------------------------------
90.6 K    Trainable params
0         Non-trainable params
90.6 K    Total params
0.363     Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.


## Tuning of hyperparameters
Here a series of sensitivity analyses are conducted to find the appropriate value of the various hyperparameters used in the model. The comparison metrics are the mean absolute error and the R-squared value both calculated based on the scaled dataset.

In [ ]:
# Set the model to evaluation mode
DFModel.eval()

# Make predictions on the test set
with torch.no_grad():
    y_pred = DFModel(X_test_rnn)
y_test_hat = y_pred.numpy()

mae = np.mean(np.abs(y_test_hat - y_test))  # Mean absolute error for the whole matrix of testing data
rmse = np.sum((y_test_hat - y_test)**2)
tss = np.sum((y_test - y_test.mean())**2)
R2 = 1 - rmse/tss
# Calculate the evaluation metrics (you can use any relevant metrics for your task)
#mae = MeanAbsoluteError()(predictions, y_test_rnn)

print(f"Mean Absolute Error on Test Set: {mae}")
print(f"R2 value on Test Set: {R2}")



Mean Absolute Error on Test Set: 0.09628978207483771
R2 value on Test Set: 0.9820485959427607


Results with various tests to check sensitivity to epochs and lag days:
    
    LagDays     Epochs   Dropout    HiddenNodes    LearningRate   MAE     R-squraed
           
      5           10       10%          128             0.001     0.159    0.9472
      8           10       10%          128             0.001     0.1489   0.9529
      10          10       10%          128             0.001     0.1473   0.9557
      3           50       10%          128             0.001     0.165    0.94
      5           50       10%          128             0.001     0.1417   0.9593
      8           50       10%          128             0.001     0.128    0.9672
      10          50       10%          128             0.001     0.1306   0.9653
      5          200       10%          128             0.001     0.141    0.9592
      8          200       10%          128             0.001     0.1272   0.9676

          

There does not seem to be a significant improvement with changing the number of epochs. However, increasing the lag days does have some effect. When the last 10 days are used are the performance decreases from when only last 8 days are used. Keeping in mind that the demand will probably cycle periodically on a weekly basis, using 7 days might lead to better results. Furthermore, increased number of hidden nodes are also checked.

Results from tests with 7 lag days:

    LagDays     Epochs   Dropout    HiddenNodes    LearningRate   MAE     R-squared
      7           10       10%          128             0.001     0.15    0.95
      7           50       10%          128             0.001     0.135   0.9635

It seems 8 days model performed better than the 7 days one, so we shall go back to that. Now we can try increasing the number of hidden nodes to check for improvements.

Sensitivity to number of hidden nodes and lag days:

    LagDays     Epochs   Dropout    HiddenNodes    LearningRate   MAE     R-squared
      5           10       10%          128             0.001     0.159    0.9472
      8           10       10%          128             0.001     0.1489   0.9529
      8           50       10%          256             0.001     0.111    0.9763
      5           50       10%          256             0.001     0.118    0.9728

The model that uses 8 days of information with 256 hidden nodes has the best observed mean absolute error of 0.111 and an R-squared value of 0.9763. Increasing the number of hidden nodes does help increase the model performance. Finally, the number of epochs and the dropout rate can also be increased along with the number of hidden nodes.

    LagDays     Epochs   Dropout    HiddenNodes    LearningRate   MAE     R-squared
      8           50       10%          256             0.001     0.111    0.9763
      8          200       10%          256             0.001     0.113    0.9754
      8           50       10%          256             0.01      0.478    0.6239
      8           50        5%          256             0.001     0.1019   0.9799
      8           50        1%          256             0.001     0.0988   0.9811
      8           50      0.5%          256             0.001     0.0963   0.9822
      8          100      0.5%          256             0.001     0.095    0.9825


The above senitivity analysis, indicates that increasing the learning rate has a negative effect on model performance. Further, reducing the dropout rate and increasing the number of epochs can improve model performance considerably. Starting with 128 hidden nodes and 5 lag days tuning the hyperparameters carefully helped increase the R-squared value of the final prediction from 94.72% to 98.25%. Thereby highlighting the importance of choosing the right hyperparameters in order to get the best performance out of the Deep Learning model.

## Comparison with demand forecast from data
The data obtained from the original source also provides a value for demand forecast for every hour. This data has been provided to ENTSOE [1] directly by the Transmission System Operators (TSOs). The demand forecast data which is a day-ahead forecast is updated regularly with every 10% change in data prediction until 2 hours before the gate closure time for the day ahead market. Although the exact model is unclear, it is based on historic load profile on similar days while also taking into account variables that affect electricity demand, such as weather, climate and socioeconomic factors [2].

This provides a yardstick to compare the performance of our model with a model used by the TSOs to decide the actual day-ahead prices. So, like the output matrix constructed in the analysis above with 24 hour demand as the columns a similar matrix can be constructed but with demand forecast as the columns. So a matrix of the same size the testing output with demand forecast value for every hour as obtained from the original source was constructed.

In [ ]:
DF = np.zeros((P,n))
for j in range(0,n):
  DF[:,j] = df["DF"].iloc[j+Lh:j+Lh+P]


DFtr,DFte = train_test_split(DF.T,test_size=0.2,random_state=42)  #Use the same seed to split. Don't forget to take transpose!
mae_dat = np.mean(np.abs(DFte - y_test))
rmse_dat = np.sum((DFte - y_test)**2)
tss = np.sum((y_test - y_test.mean())**2)
R2_dat = 1 - rmse_dat/tss
print(f"Mean Absolute Error of forecasted demand from given data on the test set: {mae_dat}")
print(f"R2 value of forecasted demand from given data on the test set: {R2_dat}")

Mean Absolute Error of forecasted demand from given data on the test set: 0.06913218112956536
R2 value of forecasted demand from given data on the test set: 0.99003860881042


On comparing the values demand forecast with the value of demand as provided in the original dataset a mean absolute error of about 0.0691 was observed with an R-squared value of 0.99. The fit of the demand forecast value in the original data is clost to 0.75% better than the fit of the model constructed in this study.

## Comparison plots
Now the standardized value are converted back into its original value and 24 hour forecasts can be compared.

In [ ]:
DF_hat = df_D_std.values*y_test_hat + np.full((y_test_hat.shape[0],y_test_hat.shape[1]),df_D_mean.values)
D = df_D_std.values*y_test + np.full((y_test_hat.shape[0],y_test_hat.shape[1]),df_D_mean.values)
DF = df_DF_std.values*DFte + np.full((y_test_hat.shape[0],y_test_hat.shape[1]),df_DF_mean.values)

You can choose which 24 period of the testing data you wish to plot and a comparison plot of the actual values, the values forecasted by the RNN model constructed above and the demand forecast values provided in the original data is developed.

Note: Since the testing data is being plotted each 24 hour period is a randomly chosen 24 hour period from the three years of data used in this study.

In [ ]:
row_index_to_plot = 0  # You can choose any row index to plot

y_df_test_row = D[row_index_to_plot,:]
y_df_hat_row = DF_hat[row_index_to_plot,:]
y_df_dat = DF[row_index_to_plot,:]

# Plot the selected row using Plotly
fig = go.Figure()


fig.add_trace(go.Scatter(x=list(range(len(y_df_test_row))), y=y_df_test_row, mode='lines', name='Actual'))
fig.add_trace(go.Scatter(x=list(range(len(y_df_hat_row))), y=y_df_hat_row, mode='lines', name='Forecast from RNN model'))
fig.add_trace(go.Scatter(x=list(range(len(y_df_dat))), y=y_df_dat, mode='lines', name='Forecast from data source'))

fig.update_layout(title=f'Comparison of actual demand with forecasted demand for row {row_index_to_plot}',
                  xaxis_title='Hourly Time Steps',
                  yaxis_title='Hourly Electricity Demand (MW)')
fig.update_layout(
    margin=dict(l=50, r=50, b=50, t=75),  # Adjust margins to make the plot tighter
    autosize=False,  # Disable autosizing
    width=1200,       # Set the width of the plot
    height=400,       # Set the height of the plot
    title_x=0.5,
    title_y=0.95
)
fig.show()

## Conclusion
In this study the electricity data from Spain was aggregated into hourly values of demand, demand forecast, generation from all sources and total import from neighbouring countries. A Recurrent Neural Network (RNN) model was developed to forecast the demand in a 24 hour period using demand, generation and import data from few days before. A sensitivity analysis based on the results of the model on testing data yielded 8 days as appropriate amount of lag with the RNN using 256 hidden units. The final model had a mean absolute error (MAE) as calculated from scaled data of about 0.095 and an R-square of 0.9825. On comparison of the results of the model with the demand forecast value provided in the original data for each of the same 24 hour periods as in the testing data of the constructed model, the values in the original data had a corresponding MAE of 0.069 and R-squared value of 0.99.  Therefore, the constructed model performs 0.75% worse than the model used by the original data to forecast the demand with the key difference is that it only uses energy demand, generation and import data not accounting for other environmental and socio-economic factors unlike the forecast provided by the transmission operator.

## References
[1] https://transparency.entsoe.eu/dashboard/show

[2] https://www.entsoe.eu/fileadmin/user_upload/_library/resources/Transparency/MoP%20Ref02%20-%20EMFIP-Detailed%20Data%20Descriptions%20V1R4-2014-02-24.pdf